In [19]:
# -*- sos -*-
###############################################################################


# 1. Generate query for healthcare facilities
[make_query_healthcare]
output: 'query_healthcare.osmql'
python: expand=True
    query = """[out:csv(::id,::type,name,amenity,operator,
         'operator:type','healthcare:speciality','addr:city','place',
         ::lat,::lon; true; ',')][timeout:1800];
    area['ISO3166-1'='PL'][admin_level=2]->.pl;
    (
      nwr(area.pl)[amenity];
      nwr(area.pl)['place'~'^(city|town|village|hamlet)$'];
    );
    out center;
    """
    with open("{_output}", "w", encoding="utf-8") as f:
        f.write(query)

###############################################################################
# 2. Execute query and download data
[healthcare_data]
input: 'query_healthcare.osmql'
output: 'query_healthcare.csv'
sh: expand=True
  curl -X POST -d @{_input} https://overpass-api.de/api/interpreter -o {_output}


###############################################################################
# 3. Analyse healthcare data and generate a multi-page PDF report
[healthcare_analysis]
input: 'query_healthcare.csv'
output: 'healthcare_analysis.pdf'
python:
    import pandas as pd
    import numpy as np
    import matplotlib.pyplot as plt
    from matplotlib.backends.backend_pdf import PdfPages
    from math import radians, sin, cos, sqrt, atan2
    import matplotlib


    def plot_heatmap(data, title, label_column='city', top_n=10):
        import matplotlib.patheffects as pe
        fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)
        hb = ax.hexbin(data['lon'], data['lat'],
                       gridsize=120, cmap='inferno',
                       bins='log', mincnt=1)
        ax.set_title(title, fontweight='bold')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')
        fig.colorbar(hb, ax=ax, label='log10(count)')
        stats = (
            data.groupby(label_column)
                .agg(count=(label_column, 'size'),
                     centroid_lat=('lat', 'mean'),
                     centroid_lon=('lon', 'mean'))
                .reset_index()
                .sort_values('count', ascending=False)
        )
        for _, row in stats.head(top_n).iterrows():
            if row[label_column] != 'unknown':
                ax.text(row.centroid_lon, row.centroid_lat, row[label_column],
                        fontsize=8, ha='center', va='center', color='lightgreen',
                        fontweight='bold',
                        path_effects=[pe.withStroke(linewidth=2, foreground='black')])
        #plt.tight_layout()
        pdf.savefig(fig)
        plt.close(fig)




    def plot_stacked_bar(df_grouped, title):
        fig, ax = plt.subplots(figsize=(10, 5), constrained_layout=True)
        df_grouped.plot(kind='bar', stacked=True, ax=ax)
        ax.set_title(title)
        ax.set_xlabel('City')
        ax.set_ylabel('Number of facilities')
        ax.tick_params(axis='x', rotation=45)
        pdf.savefig(fig)
        plt.close(fig)

    def plot_scatter(df, top_n=10):
        import matplotlib.patheffects as pe
        fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)

        for amenity, group in df.groupby('amenity'):
            ax.scatter(group['lon'], group['lat'], s=5, alpha=0.6, label=amenity)

        ax.set_title('Geographical distribution by facility type')
        ax.set_xlabel('Longitude')
        ax.set_ylabel('Latitude')

        city_counts = (
            df[df['city'] != 'unknown']
            .groupby('city')
            .size()
            .sort_values(ascending=False)
        )
        top_cities = city_counts.head(top_n).index

        centroids = (
            df[df['city'].isin(top_cities)]
            .groupby('city')
            .agg(centroid_lat=('lat', 'mean'),
                 centroid_lon=('lon', 'mean'))
            .reset_index()
        )

        for _, row in centroids.iterrows():
            ax.text(row.centroid_lon, row.centroid_lat, row['city'],
                    fontsize=7, ha='center', va='center', color='lightgreen',
                    fontweight='bold',
                    path_effects=[pe.withStroke(linewidth=1.5, foreground='black')])

        ax.legend(markerscale=3, fontsize=8)
        pdf.savefig(fig)
        plt.close(fig)


    def plot_scatter_split_by_type(df, amenities, top_n=10):
        import matplotlib.patheffects as pe
        colors = {
            'clinic': 'tab:blue',
            'hospital': 'tab:orange',
            'pharmacy': 'tab:green'
        }

        for amenity in amenities:
            subset = df[df['amenity'] == amenity]
            if subset.empty:
                continue

            fig, ax = plt.subplots(figsize=(6, 6), constrained_layout=True)
            ax.scatter(subset['lon'], subset['lat'], s=5, alpha=0.6,
                       label=amenity, color=colors.get(amenity, 'gray'))

            ax.set_title(f'{amenity.title()} – geographical distribution')
            ax.set_xlabel('Longitude')
            ax.set_ylabel('Latitude')

            city_counts = (
                subset[subset['city'] != 'unknown']
                .groupby('city').size().sort_values(ascending=False)
            )
            top_cities = city_counts.head(top_n).index

            centroids = (
                subset[subset['city'].isin(top_cities)]
                .groupby('city')
                .agg(centroid_lat=('lat', 'mean'),
                     centroid_lon=('lon', 'mean'))
                .reset_index()
            )
            for _, row in centroids.iterrows():
                ax.text(row.centroid_lon, row.centroid_lat, row['city'],
                        fontsize=7, ha='center', va='center', color='lightgreen',
                        fontweight='bold',
                        path_effects=[pe.withStroke(linewidth=1.5, foreground='black')])

            legend_lines = [f"{city}: {city_counts[city]}" for city in top_cities]
            legend_lines.append(f"\nTotal cities: {city_counts.shape[0]}")
            legend_text = "\n".join(legend_lines)

            ax.text(1.05, 1.0, legend_text,
                    transform=ax.transAxes,
                    fontsize=8, va='top', ha='left',
                    bbox=dict(boxstyle="round", fc="white", ec="gray", alpha=0.8))

            pdf.savefig(fig)
            plt.close(fig)





    EARTH_R = 6371.0  # km

    def haversine(lat1, lon1, lat2, lon2):
        dlat = radians(lat2 - lat1)
        dlon = radians(lon2 - lon1)
        a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
        return 2 * EARTH_R * atan2(sqrt(a), sqrt(1 - a))

    def plot_most_isolated_cities(med_df, sett_df, title, pdf):
        # 1. cities with hospital 
        hosp_centroids = (
            med_df[med_df['amenity'] == 'hospital']
            .groupby('city')[['lat', 'lon']]
            .mean()
            .dropna()
            .reset_index()
        )
        hosp_cities = hosp_centroids['city'].str.lower().tolist()

        # 2. cities without hospital
        deserts = sett_df[~sett_df['city'].str.lower().isin(hosp_cities)].copy()
        if deserts.empty or hosp_centroids.empty:
            return

        # vector distances
        hosp_pts = hosp_centroids[['lat', 'lon']].to_numpy()

        def nearest_dist_km(row):
            lat1, lon1 = np.radians([row.lat, row.lon])
            lats2 = np.radians(hosp_pts[:, 0])
            lons2 = np.radians(hosp_pts[:, 1])
            dlat = lats2 - lat1
            dlon = lons2 - lon1
            a = np.sin(dlat / 2) ** 2 + np.cos(lat1) * np.cos(lats2) * np.sin(dlon / 2) ** 2
            return 2 * EARTH_R * np.arctan2(np.sqrt(a), np.sqrt(1 - a)).min()

        deserts['dist_km'] = deserts.apply(nearest_dist_km, axis=1)

        # 3. TOP-10 of the farthest
        top10 = deserts.nlargest(10, 'dist_km')

        fig, ax = plt.subplots(figsize=(10, 4), constrained_layout=True)
        bars = ax.bar(top10['name'], top10['dist_km'])
        ax.set_title(title)
        ax.set_ylabel('Distance to nearest hospital city (km)')
        ax.set_xticks(range(len(top10)))
        ax.set_xticklabels(top10['name'], rotation=45, ha='right')

        for bar, val in zip(bars, top10['dist_km']):
            ax.text(bar.get_x() + bar.get_width() / 2,
                    bar.get_height() + 1,
                    f'{val:.1f} km',
                    ha='center', va='bottom', fontsize=8)

        #
        desc = ("10 Polish cities farthest from the closest city "
                "that has at least one hospital.")
        fig.text(0.01, -0.12, desc, ha='left', va='top', fontsize=8)

        pdf.savefig(fig)
        plt.close(fig)





    def plot_city_type_heatmap(df, top_n_cities=20):
        # prepare pivot table: count of facilities by amenity type and city
        tbl = (df[df["city"] != "unknown"]
               .pivot_table(index="amenity",
                            columns="city",
                            aggfunc="size",
                            fill_value=0))

        # keep only top N cities with the most total facilities
        big = tbl.sum().sort_values(ascending=False).head(top_n_cities).index
        tbl = tbl[big]

        # draw heatmap 
        fig, ax = plt.subplots(figsize=(10, 6), constrained_layout=True)
        im = ax.imshow(tbl, aspect="auto", cmap="viridis")

        # axis labels
        ax.set_xticks(range(len(tbl.columns)))
        ax.set_xticklabels(tbl.columns, rotation=45, ha="right")
        ax.set_yticks(range(len(tbl.index)))
        ax.set_yticklabels(tbl.index)

        # label each row with the max city (red outline)
        for y, row in enumerate(tbl.values):
            x = row.argmax()
            ax.text(x, y, f"{row[x]}", va="center", ha="center",
                    color="white" if row[x] > row.mean() else "black",
                    bbox=dict(boxstyle="round,pad=0.2", fc="none", ec="red", lw=1))

        fig.colorbar(im, ax=ax, label="Number of facilities")
        ax.set_title("Top cities per healthcare facility type")

        pdf.savefig(fig)
        plt.close(fig)


    def plot_bar_per_amenity(df, amenities, top_n=15):
        colors = {
            'clinic': 'tab:blue',
            'hospital': 'tab:orange',
            'pharmacy': 'tab:green'
        }

        for amenity in amenities:
            subset = df[df['amenity'] == amenity]
            city_counts = (
                subset[subset['city'] != 'unknown']
                .groupby('city').size().sort_values(ascending=False).head(top_n)
            )
            #chart
            fig, ax = plt.subplots(figsize=(10, 4), constrained_layout=True)
            city_counts.plot(kind='bar', color=colors[amenity], ax=ax)
            ax.set_title(f'{amenity.title()} – top {top_n} cities')
            ax.set_ylabel('Number of facilities')
            ax.set_xlabel('City')
            ax.tick_params(axis='x', rotation=45)

            pdf.savefig(fig)
            plt.close(fig)



    df = pd.read_csv('query_healthcare.csv')
    df = df.rename(columns={'@lat': 'lat', '@lon': 'lon'})
    df = df.dropna(subset=['lat', 'lon'])

    # Keep only selected relevant amenity types
    relevant_amenities = [
        'hospital', 'clinic', 'pharmacy',
    ]


    df['city'] = df['addr:city'].fillna('unknown')
    


    df_med = df[df['amenity'].isin(['hospital', 'clinic', 'pharmacy'])]
    df_sett = df[df['place'].notna()]

    known_df = df_med[df_med['city'] != 'unknown']

    city_amenity = (
        known_df.groupby(['city', 'amenity'])
                .size()
                .unstack(fill_value=0)
    )
    top_cities = city_amenity.sum(axis=1).sort_values(ascending=False).head(20)
    city_amenity_top = city_amenity.loc[top_cities.index]

    with PdfPages('healthcare_analysis.pdf') as pdf:
        plot_heatmap(df_med, 'Spatial density of healthcare facilities', top_n=18)
        plot_stacked_bar(city_amenity_top, 'Healthcare facilities by city (top 20 cities)')
        plot_bar_per_amenity(df_med, relevant_amenities, top_n=15)
        plot_scatter(df_med,top_n=18)
        plot_scatter_split_by_type(df_med, relevant_amenities,top_n=18)
        plot_most_isolated_cities(df_med, df_sett,'Top 10 cities farthest from a hospital', pdf)
        plot_city_type_heatmap(df_med, top_n_cities=20)
        info = pdf.infodict()
        info['Title'] = 'Healthcare facilities in Poland'
        info['Author'] = 'SoS pipeline'




In [20]:
%sosrun -t healthcare_analysis.pdf

INFO: Running make_query_healthcare : 1. Generate query for healthcare facilities

INFO: make_query_healthcare output: query_healthcare.osmql

INFO: Running healthcare_data : 2. Execute query and download data

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 73.4M    0 73.4M  100   288   344k      1  0:04:48  0:03:38  0:01:10 2009k


INFO: healthcare_data output: query_healthcare.csv

INFO: Running healthcare_analysis : 3. Analyse healthcare data and generate a multi-page PDF report

INFO: healthcare_analysis output: healthcare_analysis.pdf

INFO: Workflow default (ID=wa71d546e9dd9e493) is executed successfully with 3 completed steps.